In [ ]:
import numpy as np
import tqdm
from collections import Counter
from functools import cache

In [ ]:
file = './24.txt'
lines = open(file, 'r').readlines()
lines = [l.strip('\n') for l in lines]


In [ ]:
vals = {}
gates = []
for line in lines:
    if ':' in line:
        name, val = line.split(':')
        val = bool(int(val.strip(' ')))
        vals[name] = val
    elif '->' in line:
        splits = line.split(' ')
        name1 = splits[0]
        name2 = splits[2]
        op = splits[1]
        outname = splits[4]
        gates.append((name1,name2,op,outname))

print(len(vals))
print(vals)
print(len(gates))
print(gates[:10])


In [ ]:
from functools import cache
@cache
def comp_op(v1,v2,op):
    if op=='AND':
        return v1 and v2
    elif op=='OR':
        return v1 or v2
    elif op=='XOR':
        return v1!=v2
    else:
        raise ValueError((v1,v2,op))
    
remaining_gates = gates.copy()
all_vals = vals.copy()
n=0
while len(remaining_gates)>0 and n<200:
    n+=1
    for gate in remaining_gates:
        if gate[0] in all_vals and gate[1] in all_vals:
            res = comp_op(all_vals[gate[0]],all_vals[gate[1]],gate[2])
            all_vals[gate[3]] = res
            remaining_gates.remove(gate)


In [ ]:
xvals = []
yvals = []
zvals = []
for k, v in all_vals.items():
    try:
        kn = int(k.strip('x'))
        xvals.append((k,v))
    except:
        try:
            kn = int(k.strip('y'))
            yvals.append((k,v))
        except:
            try:
                kn = int(k.strip('z'))
                zvals.append((k,v))
            except:
                continue
xvals = sorted(xvals, reverse=True)
yvals = sorted(yvals, reverse=True)
zvals = sorted(zvals, reverse=True)
xbin = ''.join([str(int(v[1])) for v in xvals])
ybin = ''.join([str(int(v[1])) for v in yvals])
zbin = ''.join([str(int(v[1])) for v in zvals])
print(len(xvals), xvals)
print(xbin)
print(len(yvals), yvals)
print(ybin)
print(len(zvals), zvals)
print(f'current z res : {zbin} = {int(zbin,2)}')

theoric_z = str(bin(int(xbin,2) + int(ybin,2)))[2:]
print(f'correct z res : {theoric_z} = {int(theoric_z,2)}')

z_to_change = ['z'+str(len(zbin)-i-1).zfill(2) for i in range(len(theoric_z)) if theoric_z[i]!=zbin[i]]
print(f'Zxx that should change are : {z_to_change}')

all_z_names = ['z'+str(i).zfill(2) for i in range(len(zbin))]


In [ ]:
possible_swaps = [(i,j) for j in range(len(gates)) for i in range(len(gates)) if j>i]
print(len(possible_swaps))

original_vals = all_vals.copy()
change_by_swap = {}

for swap in tqdm.tqdm(possible_swaps):
    new_gates = gates.copy()
    o1 = new_gates[swap[0]][3]
    o2 = new_gates[swap[1]][3]
    new_gates[swap[0]] = (new_gates[swap[0]][0], new_gates[swap[0]][1], new_gates[swap[0]][2], o2)
    new_gates[swap[1]] = (new_gates[swap[1]][0], new_gates[swap[1]][1], new_gates[swap[1]][2], o1)
    # print(f'original gates swapped : \n{gates[swap[0]]} , {gates[swap[1]]}')
    # print(f'new gates swapped : \n{new_gates[swap[0]]} , {new_gates[swap[1]]}')
    new_vals = vals.copy()
    remaining_gates = new_gates.copy()
    n=0
    done_one = True
    while len(remaining_gates)>0 and done_one:
        n+=1
        done_one = False
        for gate in remaining_gates:
            if gate[0] in new_vals and gate[1] in new_vals:
                res = comp_op(new_vals[gate[0]],new_vals[gate[1]],gate[2])
                new_vals[gate[3]] = res
                remaining_gates.remove(gate)
                done_one = True
    if len(remaining_gates):
        # print(f'Error, for an entire cycle no gate runned!')
        # print(f'all known vals : \n{new_vals}')
        # print(f'remaining_gates : \n{remaining_gates}')
        change_by_swap[(swap[0],swap[1])] = None
    else:
        z_changed = []
        for n in all_z_names:
            if new_vals[n]!=original_vals[n]:
                z_changed.append(n)
        change_by_swap[(swap[0],swap[1])] = tuple(z_changed)


print(f'tot swap tested : {len(change_by_swap)}')
print(f'tot swap feasible : {len([ch for ch in change_by_swap.values() if ch is not None])}')
print(f'tot swap unfeasible : {len([ch for ch in change_by_swap.values() if ch is None])}')
print(f'tot swap with changes : {len([ch for ch in change_by_swap.values() if (ch is not None) and len(ch)>0])}')



In [ ]:
feasible_swaps = {k: ch for k, ch in change_by_swap.items() if ch is not None}
feasible_swaps = {k: ch for k, ch in feasible_swaps.items() if len(ch)>0}
swaps_per_z = {}
perfect_swaps_per_z = {}
for zerr in z_to_change:
    zswaps = [k for k, v in feasible_swaps.items() if zerr in v]
    perfect_zswaps = [k for k in zswaps if set(feasible_swaps[k]).issubset(z_to_change)]
    print(f'looking at {zerr}, swaps that change it are : {len(zswaps)}')
    print(f'swaps that change only zs to change : {len(perfect_zswaps)}')
    swaps_per_z[zerr] = zswaps    
    perfect_swaps_per_z[zerr] = perfect_zswaps

In [ ]:
def find_next_gates(gates, i, l=2):
    o = gates[i][3]
    gsdict = {}
    gs = [g for g in gates if g[0]==o or g[1]==o]
    gsdict[1] = gs.copy()
    if l==2:
        gs2 = []
        for g in gsdict[1]:
            o = g[3]
            gs = [g for g in gates if g[0]==o or g[1]==o]
            gs2.extend(gs)
        gsdict[2] = gs2
    return gsdict

swap = (10,19)
print(f'Swap {swap} change {feasible_swaps[swap]}')
print(f'gate {swap[0]} is {gates[swap[0]]}')
print(f'gate {swap[1]} is {gates[swap[1]]}')
print(f'nexts gate {swap[0]} -> {find_next_gates(gates, swap[0])}')
print(f'nexts gate {swap[1]} -> {find_next_gates(gates, swap[1])}')

o1_nexts = [g for g in gates if g[0]==gates[swap[0]][3] or g[1]==gates[swap[0]][3]]

swap = (10,71)
print(f'Swap {swap} change {feasible_swaps[swap]}')
print(f'gate {swap[0]} is {gates[swap[0]]}')
print(f'gate {swap[1]} is {gates[swap[1]]}')
print(f'nexts gate {swap[0]} -> {find_next_gates(gates, swap[0])}')
print(f'nexts gate {swap[1]} -> {find_next_gates(gates, swap[1])}')

In [ ]:
z_to_change = sorted(z_to_change)
print(f'LOOKING FOR SWAPS :')
print(f'ALL ZS TO CHANGE : {z_to_change}')

choosen_swaps = [
    (105,142),
    (71, 187),
    (120, 220),
    (72, 116)
    # (64,173),
    # (28,220),
    # (72,177)
    # (4, 141),
    # (28, 220),
    # (72,116),
    # (13,72),
]
for chsw in choosen_swaps:
    print(f'SWAP {chsw} gives {feasible_swaps[chsw]}')
z_to_exclude = []
for chsw in choosen_swaps:
    z_to_exclude.extend(list(feasible_swaps[chsw]))
missing_zs = [z for z in z_to_change if z not in z_to_exclude]
print(f'Zs still to change {missing_zs}')
if len(missing_zs)>0:
    z_to_look = missing_zs[0]
    print(f'now looking to change {z_to_look}')
    for swap in perfect_swaps_per_z[z_to_look]:
        already=False
        for z in z_to_exclude:
            if z in feasible_swaps[swap]:
                already = True
        if not already:
            print(f'swap {swap} gives {feasible_swaps[swap]}')

In [ ]:
z_th_change = []
for ch in choosen_swaps:
    z_th_change.extend(list(feasible_swaps[ch]))
print(f'Checking the swaps : {choosen_swaps}')
print(f'Concatenating result change : {z_th_change}')
z_cnt = {o: n for o,n in Counter(z_th_change).items()}
z_cnt = {o: z_cnt[o] for o in sorted(z_cnt.keys())}
print(f'changes counter = {z_cnt}')

changed_gates = gates.copy()
for swap in choosen_swaps:
    o1 = changed_gates[swap[0]][3]
    o2 = changed_gates[swap[1]][3]
    changed_gates[swap[0]] = (changed_gates[swap[0]][0], changed_gates[swap[0]][1], changed_gates[swap[0]][2], o2)
    changed_gates[swap[1]] = (changed_gates[swap[1]][0], changed_gates[swap[1]][1], changed_gates[swap[1]][2], o1)
    # print(f'original gates swapped : \n{gates[swap[0]]} , {gates[swap[1]]}')
    # print(f'new gates swapped : \n{new_gates[swap[0]]} , {new_gates[swap[1]]}')
new_vals = vals.copy()
remaining_gates = changed_gates.copy()
n=0
done_one = True
while len(remaining_gates)>0 and done_one:
    n+=1
    done_one = False
    for gate in remaining_gates:
        if gate[0] in new_vals and gate[1] in new_vals:
            res = comp_op(new_vals[gate[0]],new_vals[gate[1]],gate[2])
            new_vals[gate[3]] = res
            remaining_gates.remove(gate)
            done_one = True
if len(remaining_gates):
    # print(f'Error, for an entire cycle no gate runned!')
    # print(f'all known vals : \n{new_vals}')
    # print(f'remaining_gates : \n{remaining_gates}')
    change_by_swap[(swap[0],swap[1])] = None
    print(f'Cannot complete the result!')
else:
    z_changed = []
    for n in all_z_names:
        if new_vals[n]!=original_vals[n]:
            z_changed.append(n)
print(f'Resulting change instead is {z_changed}')
if set(z_changed) == set(z_to_change):
    print(f'FOUND SOLUTION!!!!')

    sol_wires = [(gates[sw[0]][3], gates[sw[1]][3]) for sw in choosen_swaps]
    sol_wires = [w for wtuple in sol_wires for w in wtuple]
    print(f'solution wires are : {sol_wires}')
    print(f'sorted : {sorted(sol_wires)}')
    print(f'SOL: \n{",".join(sorted(sol_wires))}')

In [ ]:
import sys
from typing import Iterator, NamedTuple


class GateExpression(NamedTuple):
    label: str
    position: int


class Mismatch(NamedTuple):
    expr1: "GateExpression | Mismatch"
    expr2: "GateExpression | Mismatch"
    gate: str
    position: int


def parse_input(lines: Iterator[str]) -> tuple[dict[str, int], list[tuple[str, str, str, str]]]:
    inputs = {}
    gates = []
    part = 1
    for line in lines:
        line = line.strip()
        if line == "":
            part = 2
            continue
        if part == 1:
            wire, value = line.split(": ")
            inputs[wire] = int(value)
        else:
            wire1, gate, wire2, _, output = line.split()
            gates.append((wire1, gate, wire2, output))
    return inputs, gates


def simulate_gates(inputs: dict, gates: list):
    wire_values = inputs.copy()
    waiting_z_wires = {output for _, _, _, output in gates if output.startswith("z")}
    while waiting_z_wires:
        for wire1, gate, wire2, output in gates:
            if wire1 in wire_values and wire2 in wire_values:
                w1 = wire_values[wire1]
                w2 = wire_values[wire2]
                if gate == "AND":
                    result = w1 & w2
                elif gate == "OR":
                    result = w1 | w2
                elif gate == "XOR":
                    result = w1 ^ w2
                else:
                    raise
                wire_values[output] = result
                if output in waiting_z_wires:
                    waiting_z_wires.remove(output)
    return wire_values


def get_output_value(wire_values: dict[str, int]) -> int:
    result = 0
    for wire, value in wire_values.items():
        if value and wire.startswith("z"):
            place = int(wire[1:])
            result += 1 << place
    return result


def find_swaps(gates: list[tuple[str, str, str, str]]) -> Iterator[tuple[str, str]]:
    while True:
        swap = find_swap(gates)
        if swap:
            yield swap
            swap_outputs(swap[0], swap[1], gates)
        else:
            break


def find_swap(gates: list[tuple[str, str, str, str]]) -> tuple[str, str]:
    gates_by_output = {output: (wire1, gate, wire2) for wire1, gate, wire2, output in gates}
    expressions = {get_expr(output, gates_by_output): output for output in gates_by_output}
    for expr, output in expressions.items():
        expr = get_expr(output, gates_by_output)
        if expr.position == -1 and output.startswith("z"):
            expr1, expr2, gate, _ = expr
            position = int(output[1:])
            if gate == "XOR":
                wire1, _, wire2 = gates_by_output[output]
                other_wire = {expr: wire for expr, wire in zip((expr1, expr2), (wire2, wire1))}
                if GateExpression("Carry", position) in (expr1, expr2):
                    return (
                        other_wire[GateExpression("Carry", position)],
                        expressions[GateExpression("Xor", position)],
                    )
                if GateExpression("Xor", position) in (expr1, expr2):
                    return (
                        other_wire[GateExpression("Xor", position)],
                        expressions[GateExpression("Carry", position)],
                    )
        elif expr.position < 0:
            continue
        elif expr.label == "Digit" and output != f"z{expr.position:02d}":
            return output, f"z{expr.position:02d}"


def get_expr(output, gates_by_output) -> GateExpression | Mismatch:
    """
        Digit(0) = Xor(0)
        Digit(N) = Carry(N) XOR Xor(N)
        Carry(1) = And(0)
        Carry(N+1) = Partial(N) OR And(N)
        Partial(N) = Carry(N) AND Xor(N)
        Xor(N) = x(N) XOR y(N)
        And(N) = x(N) AND y(N)
    """
    if output[0] in "xy":
        return GateExpression(output[0], int(output[1:]))
    wire1, gate, wire2 = gates_by_output[output]
    expr1 = get_expr(wire1, gates_by_output)
    expr2 = get_expr(wire2, gates_by_output)
    if expr1.position == expr2.position and (not isinstance(expr1, Mismatch)) and (not isinstance(expr2, Mismatch)) :
        labels = {expr1.label, expr2.label}
        if labels == {"x", "y"}:
            if gate == "XOR":
                return GateExpression("Xor" if expr1.position > 0 else "Digit", expr1.position)
            elif gate == "AND":
                if expr1.position == 0:
                    return GateExpression("Carry", 1)
                else:
                    return GateExpression("And", expr1.position)
        if labels == {"Carry", "Xor"} and gate in ("AND", "XOR"):
            return GateExpression("Partial" if gate == "AND" else "Digit", expr1.position)
        if labels == {"Partial", "And"} and gate == "OR":
            return GateExpression("Carry", expr1.position + 1)
    if expr1.position < 0 or expr2.position < 0:
        return Mismatch(expr1, expr2, gate, -2)
    return Mismatch(expr1, expr2, gate, -1)


def swap_outputs(output1, output2, gates: list[tuple[str, str, str, str]]) -> None:
    for i, (wire1, gate, wire2, output) in enumerate(gates):
        if output == output1:
            new_output = output2
        elif output == output2:
            new_output = output1
        else:
            new_output = output
        gates[i] = (wire1, gate, wire2, new_output)


def main():
    file = './24.txt'
    lines = open(file, 'r').readlines()
    lines = [l.strip('\n') for l in lines]
    inputs, gates = parse_input(lines)
    print(get_output_value(simulate_gates(inputs, gates)))
    swap1, swap2, swap3, swap4 = find_swaps(gates)
    print(swap1, swap2, swap3, swap4)
    print(",".join(sorted(swap1 + swap2 + swap3 + swap4)))


if __name__ == '__main__':
    main()